
### Key States:
1. **Start**: Initialize new sorting pass
2. **Compare**: Check adjacent elements
3. **Swap**: Exchange elements if out of order
4. **Check Flag**: Determine if sorting is complete

### Transitions:
- `Compare → Swap` when current element > next element
- `Compare → Check Flag` when end of list reached
- `Swap → Compare` after performing swap
- `Check Flag → Start` if swaps occurred (flag=1)
- `Check Flag → Halt` if no swaps (flag=0)



```mermaid
stateDiagram-v2
    [*] --> Start
    Start --> Compare: New pass\nReset head
    Compare --> Swap: element > next
    Compare --> Check_Flag: End reached
    Swap --> Compare: Swap done\nMove right
    Check_Flag --> Start: Flag=1
    Check_Flag --> [*]: Flag=0
```

In [4]:
class TuringMachine:
    def __init__(self, tape):
        self.tape = tape.copy() + ['0']  #Faz uma cópia da fita adicionando um 0 ao final para indicar um flag de troca
        self.head = 0 # Posição inicial da cabeça de leitura
        self.state = 'start' #Estado inicial
        self.swap_flag_pos = len(self.tape) - 1 #Armazenar a posição do flag de troca
        self.history = []  
        self.step_counter = 0

    def log_action(self, action):
        self.step_counter += 1
        self.history.append(f"Step {self.step_counter}: {action}")

    def step(self): # Se a maquina ja estiver parada, não faz nada
        if self.state == 'halt':
            return

        current_symbol = self.tape[self.head] if self.head < len(self.tape) else None

        if self.state == 'start': # Inicia a maquina na posição 0 resetando a cabeça de leitura
            action = f"START NEW PASS | Reset head to position 0"
            self.head = 0
            self.state = 'compare'
            self.log_action(action)

        elif self.state == 'compare': # Se a cabeça de leitura estiver no ultimo elemento, verifica se a ordenação foi concluida
            if self.head >= self.swap_flag_pos - 1:
                action = "Reached end of elements | Check swap flag"
                self.state = 'check_flag'
                self.log_action(action)
            else: #Compara o simbolo atual com o proximo
                next_symbol = self.tape[self.head + 1]
                if current_symbol > next_symbol: #Se o simbolo atual for maior que o proximo, a maquina entra no estado de troca
                    action = f"COMPARE [{current_symbol}↔{next_symbol}] at positions {self.head}-{self.head+1} | Need swap"
                    self.state = 'swap'
                    self.log_action(action)
                else: #Se o simbolo atual for menor que o proximo, a maquina avança para o proximo simbolo
                    action = f"COMPARE [{current_symbol}↔{next_symbol}] at positions {self.head}-{self.head+1} | No swap"
                    self.head += 1
                    if self.head >= self.swap_flag_pos - 1: #Se a cabeça de leitura estiver no ultimo elemento, transiciona para o estado de verificação do flag
                        self.state = 'check_flag'
                        action += " | Reached end"
                    self.log_action(action)

        elif self.state == 'swap': #Troca os elementos de posição caso necessário
            prev_val = self.tape[self.head]
            next_val = self.tape[self.head + 1]
            self.tape[self.head], self.tape[self.head + 1] = next_val, prev_val
            self.tape[self.swap_flag_pos] = '1' #Define o sinalizador de troca como '1', indicando que pelo menos uma troca ocorreu nesta passagem
            
            action = f"SWAP [{prev_val}⇄{next_val}] at positions {self.head}-{self.head+1} | "
            self.head += 1 #Move a cabeça de leitura para o proximo elemento
            action += f"Moved to position {self.head} | Set swap flag"
            
            if self.head >= self.swap_flag_pos - 1:  #Se a cabeça de leitura estiver no ultimo elemento, transiciona para o estado de verificação do flag
                self.state = 'check_flag'
                action += " | Reached end"
            else:
                self.state = 'compare'
                
            self.log_action(action)

        elif self.state == 'check_flag': # se o sinalizador de troca for 1, reinicia a maquina, caso contrario, a ordenação foi concluida
            if self.tape[self.swap_flag_pos] == '1':
                action = "CHECK FLAG [1] | Swaps occurred | Reset flag and restart"
                self.tape[self.swap_flag_pos] = '0'
                self.state = 'start'
            else: #Se o sinalizador de troca for 0, a ordenação foi concluida
                action = "CHECK FLAG [0] | No swaps | Sorting complete"
                self.state = 'halt'
            self.log_action(action)

    def run(self): #Executa a maquina ate que o estado seja 'halt'
        while self.state != 'halt':
            self.step()
        return self.tape[:-1], self.history

initial_elements = ['1', '4', '4','3','2']
tm = TuringMachine(initial_elements)
sorted_tape, history = tm.run()

print("Initial tape:", initial_elements)
print("Sorted tape:", sorted_tape)
print("\nExecution log:")
print("\n".join(history))


Initial tape: ['1', '4', '4', '3', '2']
Sorted tape: ['1', '2', '3', '4', '4']

Execution log:
Step 1: START NEW PASS | Reset head to position 0
Step 2: COMPARE [1↔4] at positions 0-1 | No swap
Step 3: COMPARE [4↔4] at positions 1-2 | No swap
Step 4: COMPARE [4↔3] at positions 2-3 | Need swap
Step 5: SWAP [4⇄3] at positions 2-3 | Moved to position 3 | Set swap flag
Step 6: COMPARE [4↔2] at positions 3-4 | Need swap
Step 7: SWAP [4⇄2] at positions 3-4 | Moved to position 4 | Set swap flag | Reached end
Step 8: CHECK FLAG [1] | Swaps occurred | Reset flag and restart
Step 9: START NEW PASS | Reset head to position 0
Step 10: COMPARE [1↔4] at positions 0-1 | No swap
Step 11: COMPARE [4↔3] at positions 1-2 | Need swap
Step 12: SWAP [4⇄3] at positions 1-2 | Moved to position 2 | Set swap flag
Step 13: COMPARE [4↔2] at positions 2-3 | Need swap
Step 14: SWAP [4⇄2] at positions 2-3 | Moved to position 3 | Set swap flag
Step 15: COMPARE [4↔4] at positions 3-4 | No swap | Reached end
Step 16: C

## 2. Transition Table

| Current State | Read Symbol | Next State  | Write Symbol | Move Direction | Condition/Description      |
|--------------|------------|-------------|--------------|---------------|----------------------------|
| start        | *          | compare     | same         | R             | Begin new pass             |
| compare      | a          | compare     | a            | R             | a ≤ next element           |
| compare      | a          | swap        | a            | -             | a > next element           |
| swap         | a          | compare     | b            | R             | Swap a↔b, set flag         |
| compare      | ☐         | check_flag  | ☐           | L             | End of elements            |
| check_flag   | 0          | halt        | 0            | -             | No swaps → sorted          |
| check_flag   | 1          | start       | 0            | L             | Reset flag → new pass      |

### Where:
- `a, b` ∈ input alphabet (e.g., digits 0-9)
- `☐` = blank symbol (end marker)
- `R / L` = move right/left


Step | State       | Tape (Head Position)       | Action
-----|-------------|-----------------------------|---------------------------
1    | start       | [3̲,1,2,4,0]                 | Reset head
2    | compare     | [3̲,1,2,4,0]                 | Compare 3 > 1 → swap
3    | swap        | [1̲,3,2,4,1]                 | Swap completed
4    | compare     | [1,3̲,2,4,1]                 | Compare 3 > 2 → swap
5    | swap        | [1,2̲,3,4,1]                 | Swap completed
6    | compare     | [1,2,3̲,4,1]                 | Compare 3 < 4 → move
7    | check_flag  | [1,2,3,4,1̲]                | Flag=1 → new pass
8    | start       | [1̲,2,3,4,0]                 | Reset for next pass
9    | compare     | [1̲,2,3,4,0]                 | All elements sorted → halt

## 3. Formal 7-Tuple Definition

```python
TM = (Q, Σ, Γ, δ, q₀, q_accept, q_reject)
Q (States): {start, compare, swap, check_flag, halt}
Σ (Input Alphabet): {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Γ (Tape Alphabet): Σ ∪ {0, 1, ☐} (Includes swap flags)
δ (Transition Function): As defined in the transition table
q₀ (Start State): start
q_accept: halt (No separate accept/reject states needed for sorting)
q_reject: Not used (Always halts with output)